In [2]:
import tensorflow.keras as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics
import tf2onnx
import numpy as np
import math
from pathlib import Path
from keras.utils.np_utils import to_categorical

In [3]:
n_classes = 10
nb_classes = 10
input_shape = (28, 28) #Lenet5
#batch_size = 500 # Lenet
batch_size = 500 # Lenet
input_shape = (28, 28) #Lenet5

In [4]:
img_rows, img_cols = 64, 64
(x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()
#x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32') 
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_test)


print((x_train.shape,y_train.shape))
#print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((50000, 32, 32, 3), (50000, 10))
((10000, 32, 32, 3), (10000, 10))


In [5]:
training_id = 1
model_short_name = 'resnet18'
framework = 'keras'

In [6]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/{}/{}/'.format(framework, model_short_name)
since_0 = time.time()
#model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
#model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
model_name = 'tf_resnet18-cifar10_2021-10-29_{}'.format(training_id)
model = tf.keras.models.load_model(path+ model_name+'.h5')
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+ model_name+'.h5')
size0

print(t_elapsed_0)

1.3454258441925049


In [7]:
import onnx
import onnxruntime
import coremltools
import time

In [8]:
onnx_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/'
coreml_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/coremltools/'
error_path = '/Volumes/Cisco/Fall2021/onnx-exchange/miss-classification/v2/'

In [9]:
def model_scores(y_test, test_predict):
    correct_ = np.sum(y_test == test_predict)
    accuracy  = correct_*100./np.sum(y_test == y_test)
    return accuracy
def get_miss_classification(y1, y2):
    miss_perc_val_original_coreml = 0
    try:
        np.testing.assert_array_equal(y1, y2)
    except Exception as e:
        miss_perc_val_original_coreml = 1
    return miss_perc_val_original_coreml #, precision2, recall2, f12

In [10]:
def to_onnx(i, x, y, batch_size):
    
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
    #print("converting for batch: ", i)
    
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    
    ### Original Model
    since_1 = time.time()
    #model = torch.load(path+model_name+'.pth')
    with tf.device('/cpu:0'): 
        k_predict = model.predict(x)
    inference_time_original = time.time() - since_1
    y0 = to_categorical(np.argmax(k_predict, 1), num_classes = 10)
    correct_original = np.sum(y0 == y)
    accuracy_original = model_scores(y, y0)
    miss_test_original = get_miss_classification(y, y0)
    # ONNX Model
    
    t_elapsed_2 = time.time() - since_1
    #since_1 = time.time()
    #onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #load_time_onnx = time.time() - since_1
    #onnx.checker.check_model(onnx_model)
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    #ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: x}
    ort_outs = ort_session.run(None, ort_inputs)
    inference_time_onnx = time.time() - since_1
    # compare ONNX Runtime and PyTorch results
    #print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    ####### Mis-classification ONNX ######################################
    y2 = to_categorical(np.argmax(ort_outs[0], 1), num_classes = 10)
    correct_onnx = np.sum(y2 == y)
    accuracy_onnx = model_scores(y, y2)
    miss_original_onnx = get_miss_classification(y0, y2)
    miss_test_onnx = get_miss_classification(y, y2)
    ####### End of mis-classification ONNX ###################################### 
    
    
    ## CoreML
    
    #since_1 = time.time()
    #coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}.mlmodel".format(model_short_name, model_name))
    #load_time_coreml = time.time() - since_1
    
    #spec = coreml_model.get_spec()
    #coreml_model = coremltools.models.MLModel(spec)
    split_ = str(coreml_model.get_spec().description.input[0]).split('\n')
    name_1 = split_[0].replace('name: "', '')
    name_1 = name_1.replace('"', '')
    
    since_1 = time.time()
    output_dict_test = coreml_model.predict({name_1:x})
    inference_time_coreml = time.time() - since_1
    ####### Mis-classification coreML ######################################
    y3 = to_categorical(np.argmax(output_dict_test['Identity'], 1), num_classes = 10)
    correct_coreml = np.sum(y3 == y)
    accuracy_coreml = model_scores(y, y3)
    
    #print(correct_original, correct_coreml, correct_onnx, np.sum(y == y))
    ## Part 1
    
    miss_original_coreml = get_miss_classification(y0, y3)
    miss_test_coreml = get_miss_classification(y, y3)
    #return correct_original,correct_onnx,correct_coreml 
    list_val = [accuracy_original, accuracy_onnx, accuracy_coreml, miss_test_original, miss_original_onnx, miss_test_onnx, miss_original_coreml, miss_test_coreml,'', inference_time_original, inference_time_onnx, inference_time_coreml]
    return list_val

In [11]:
def _lets_convert(x, y, data_writer_run, batch_size): # for cifar10 etc
    since = time.time()
    batches = 0
    since = time.time()
    accuracy_original = 0.0
    accuracy_onnx = 0.0
    accuracy_coreml = 0.0
    miss_test_original = 0
    miss_original_onnx = 0
    miss_test_onnx = 0
    miss_original_coreml = 0
    miss_test_coreml = 0
    
    inference_time_original = 0.0
    inference_time_onnx = 0.0
    inference_time_coreml = 0.0
    
    total_ = 0.0
    total_datasets = 0
    for batches in range(len(x)):
        x_batch = x[batches].reshape(1, 32, 32, 3)
        y_batch = y[batches].reshape(1, 10)
        list_val = to_onnx(batches, x_batch, y_batch, batch_size)
        batches += 1
        if batches%50 == 0:
            print(batches, list_val)
        accuracy_original += list_val[0]
        accuracy_onnx += list_val[1]
        accuracy_coreml += list_val[2]
        
        miss_test_original += list_val[3]
        miss_original_onnx += list_val[4]
        miss_test_onnx += list_val[5]
        miss_original_coreml += list_val[6]
        miss_test_coreml += list_val[7]
        
        inference_time_original += list_val[9]
        inference_time_onnx += list_val[10]
        inference_time_coreml += list_val[11]
        total_ += np.sum(y_batch == y_batch)
        total_datasets += y_batch.shape[0]
        if batches == 1000:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )
    #print(correct_original, correct_onnx, correct_coreml, total, correct_original*100./total, precision2, recall2, f1)
    #return correct_original*100./total, correct_onnx*100./total, correct_coreml*100./total
    data_writer_run.writerow([model_short_name,framework, training_id, model_name, batch_size, total_,total_datasets, round(accuracy_original*100/total_,2), round(accuracy_onnx*100/total_,2), round(accuracy_coreml*100/total_,2), round(miss_test_original*100/total_datasets,2), round(miss_original_onnx*100/total_datasets,2), round(miss_test_onnx*100/total_datasets,2), round(miss_original_coreml*100/total_datasets,2), round(miss_test_coreml*100/total_datasets,2),'', inference_time_original/total_datasets, inference_time_onnx/total_datasets, inference_time_coreml/total_datasets, '',
                              '{:.0f}m {:.0f}s'.format((inference_time_original/total_datasets) // 60, (inference_time_original/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format((inference_time_onnx/total_datasets) // 60, (inference_time_onnx/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format((inference_time_coreml/total_datasets) // 60, (inference_time_coreml/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)])

In [12]:
import pandas as pd 
if not os.path.exists(error_path+framework):
    Path(error_path+framework).mkdir(parents=True, exist_ok=True)
#data_file_run = open(error_path+framework+"/{}/runtime_miss-classification_{}.csv".format(model_short_name,model_name), mode='w', newline='',
#                                  encoding='utf-8')
#data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','runtime','original_load_time', 'original_infererence_time', 'runtime_load_time', 
#                          'runtime_inference_time',  'miss_classified_original_runtime_percentage','',  'encoded_miss_classified_original_runtime_percentage','encoded_miss_classified_original_test_runtime_percentage', '', 'accuracy_original', 'accuracy_runtime'])

for round_ in [1,2,3,4,5,6,7,8,9,10]:
    training_id = round_
    model_short_name = 'resnet18'
    framework = 'keras'
    path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/{}/{}/'.format(framework, model_short_name)
    since_0 = time.time()
    #model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
    #model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
    model_name = 'tf_resnet18-cifar10_2021-10-29_{}'.format(training_id)
    #model_name = 'tf_vgg-cifar10_2021-10-29_{}'.format(training_id)
    model = tf.keras.models.load_model(path+ model_name+'.h5')
    t_elapsed_0 = time.time() - since_0
    size0 = os.path.getsize(path+ model_name+'.h5')
    size0
    
    since_1 = time.time()
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    load_time_onnx = time.time() - since_1
    onnx.checker.check_model(onnx_model)
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    
    
    since_1 = time.time()
    coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}.mlmodel".format(model_short_name, model_name))
    load_time_coreml = time.time() - since_1
    
    print('round: ', round_)
    flag = 0
    if not os.path.exists(error_path+framework+"/runtime_accuracy_{}.csv".format(model_short_name)):
        data_file_run2 = open(error_path+framework+"/runtime_accuracy_{}.csv".format(model_short_name), mode='w', newline='', encoding='utf-8')
    else:
        data_file_run2 = open(error_path+framework+"/runtime_accuracy_{}.csv".format(model_short_name), mode='a+', newline='', encoding='utf-8')
        flag = 1
    data_writer_run2 = csv.writer(data_file_run2, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    if flag == 0: 
        data_writer_run2.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'test_set','execution_times','accuracy_original', 'accuracy_onnx', 'accuracy_coreml', 'miss_test_original', 'miss_original_onnx', 'miss_test_onnx', 'miss_original_coreml', 'miss_test_coreml','', 'inference_time_original', 'inference_time_onnx', 'inference_time_coreml', '', 'inference_time_original2', 'inference_time_onnx2', 'inference_time_coreml2', 'overral_time'])


    _lets_convert(x_test, y_test,  data_writer_run2, batch_size)
    
    data_file_run2.close()
#data_file_acc.close()

round:  1
50 [100.0, 100.0, 100.0, 0, 0, 0, 0, 0, '', 0.06113028526306152, 0.010437965393066406, 0.02211904525756836]
100 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.06153678894042969, 0.011238813400268555, 0.022210121154785156]
150 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.07022213935852051, 0.011703014373779297, 0.021027088165283203]
200 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.07272505760192871, 0.010864973068237305, 0.04320120811462402]
250 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.10453391075134277, 0.02193927764892578, 0.02228999137878418]
300 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.08905887603759766, 0.018585920333862305, 0.020925045013427734]
350 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.08810710906982422, 0.019282102584838867, 0.021966934204101562]
400 [100.0, 100.0, 100.0, 0, 0, 0, 0, 0, '', 0.08081912994384766, 0.019530057907104492, 0.02175426483154297]
450 [100.0, 100.0, 100.0, 0, 0, 0, 0, 0, '', 0.1416759490966797, 0.06113100051879883, 0.02253103256225586]
500 [80.0, 80.0, 80.0, 1, 0

850 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.33373594284057617, 0.09395289421081543, 0.016294240951538086]
900 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.12648487091064453, 0.07465481758117676, 0.025516748428344727]
950 [100.0, 100.0, 100.0, 0, 0, 0, 0, 0, '', 0.22179031372070312, 0.05261588096618652, 0.01942920684814453]
1000 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.15428996086120605, 0.029091835021972656, 0.026359081268310547]
Conversion complete in 34m 28s
round:  5
50 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.12310123443603516, 0.020792007446289062, 0.023708105087280273]
100 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.5045771598815918, 0.20186495780944824, 0.019414901733398438]
150 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.9055368900299072, 0.18704581260681152, 0.01641702651977539]
200 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.13607096672058105, 0.021603107452392578, 0.04559063911437988]
250 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.14366912841796875, 0.03457808494567871, 0.07039904594421387]
300 [

650 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.12573695182800293, 0.03333473205566406, 0.023331880569458008]
700 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.12687110900878906, 0.03665280342102051, 0.02302694320678711]
750 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.1077570915222168, 0.02270793914794922, 0.05776810646057129]
800 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.11049604415893555, 0.02840590476989746, 0.02237701416015625]
850 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.10455918312072754, 0.02160501480102539, 0.025371074676513672]
900 [100.0, 100.0, 100.0, 0, 0, 0, 0, 0, '', 0.09913992881774902, 0.019010066986083984, 0.022336959838867188]
950 [100.0, 100.0, 100.0, 0, 0, 0, 0, 0, '', 0.10399007797241211, 0.02262091636657715, 0.020994901657104492]
1000 [100.0, 100.0, 100.0, 0, 0, 0, 0, 0, '', 0.09643220901489258, 0.018504858016967773, 0.02214193344116211]
Conversion complete in 15m 58s
round:  9
50 [80.0, 80.0, 80.0, 1, 0, 1, 0, 1, '', 0.10567808151245117, 0.021782875061035156, 0.022689104080200195

In [10]:
x_test.shape

(10000, 32, 32, 3)

In [15]:
for i in range(len(x_test)):
    print(x_test[i].reshape(1, 32, 32, 3).shape, y_test[i].reshape(1, 10).shape)
    break

(1, 32, 32, 3) (1, 10)


In [12]:
len(x_test)

10000

In [13]:
model_short_name = 'vgg'
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/{}/{}/'.format(framework, model_short_name)
since_0 = time.time()
#model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
#model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
model_name = 'tf_vgg-cifar10_2021-10-29_{}'.format(training_id)
model = tf.keras.models.load_model(path+ model_name+'.h5')
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+ model_name+'.h5')
size0

71945464